# **DataSet Generation For Food and Water**

In [1]:
import pandas as pd
import numpy as np

# Constants for food and water supply
RICE_PER_PERSON_PER_DAY = 0.4  # kg
VEGETABLES_PER_PERSON_PER_DAY = 0.2  # kg
DRY_FOOD_PER_PERSON_PER_DAY = 0.15  # kg
WATER_PER_PERSON_PER_DAY = 3.785  # liters (1 gallon)

# Generate synthetic dataset for food and water supply
def generate_food_water_data(num_samples=1000):
    np.random.seed(42)
    data = []

    for _ in range(num_samples):
        disaster_type = np.random.choice(['Flood', 'Storm', 'Earthquake', 'Drought'])
        severity = np.random.randint(1, 6)  # Scale 1-5
        area_size = np.random.randint(10, 5000)  # in square km
        population_affected = np.random.randint(100, 50000)
        duration = np.random.randint(1, 30)  # in days

        # Calculate food and water supply
        total_rice = population_affected * duration * np.random.uniform(0.9, 1.1) * RICE_PER_PERSON_PER_DAY  # kg
        total_vegetables = population_affected * duration * np.random.uniform(0.9, 1.1) * VEGETABLES_PER_PERSON_PER_DAY  # kg
        total_dry_food = population_affected * duration * np.random.uniform(0.9, 1.1) * DRY_FOOD_PER_PERSON_PER_DAY  # kg
        total_water = population_affected * duration * np.random.uniform(0.9, 1.1) * WATER_PER_PERSON_PER_DAY

        data.append([disaster_type, severity, area_size, population_affected, duration,
                     total_rice, total_vegetables, total_dry_food, total_water])

    df = pd.DataFrame(data, columns=["Disaster Type", "Severity", "Area Size (sq km)", "Population Affected",
                                     "Duration (days)", "Rice Supply (kg)", "Vegetables Supply (kg)",
                                     "Dry Food Supply (kg)", "Water Supply (liters)"])
    return df

# Generate and save datasets
food_water_df = generate_food_water_data()
food_water_df.to_csv("synthetic_food_water_data.csv", index=False)

print("Datasets generated and saved successfully.")


Datasets generated and saved successfully.


# **DataSet Generation For Medicine and Clothing**



In [2]:
import pandas as pd
import numpy as np

# Generate synthetic dataset for medicine and clothing supply
def generate_medicine_clothing_data(num_samples=1000):
    np.random.seed(42)
    data = []

    for _ in range(num_samples):
        disaster_type = np.random.choice(['Flood', 'Storm', 'Earthquake', 'Drought'])
        severity = np.random.randint(1, 6)
        area_size = np.random.randint(10, 5000)
        population_affected = np.random.randint(100, 50000)
        duration = np.random.randint(1, 30)

        # Age and gender distribution
        age_0_12 = np.random.uniform(0.1, 0.3) * population_affected
        age_12_60 = np.random.uniform(0.5, 0.7) * population_affected
        age_60_plus = population_affected - (age_0_12 + age_12_60)
        female_ratio = np.random.uniform(0.45, 0.55)
        females = female_ratio * population_affected

        # Calculate supplies
        baby_food = min(age_0_12 * duration * np.random.uniform(0.9, 1.1) * 0.2, 5000)  # kg
        elder_medicine = min(age_60_plus * duration * np.random.uniform(0.9, 1.1), 10000)  # units
        sanitary_items = min(females * duration * np.random.uniform(0.9, 1.1) * 0.5, 8000)  # units

        clothing_fraction = np.random.uniform(0.2, 0.5)
        clothing_supply = min(population_affected * clothing_fraction, 5000)

        data.append([disaster_type, severity, area_size, population_affected, duration,
                     age_0_12, age_12_60, age_60_plus, females, baby_food, elder_medicine,
                     sanitary_items, clothing_supply])

    df = pd.DataFrame(data, columns=["Disaster Type", "Severity", "Area Size (sq km)", "Population Affected",
                                     "Duration (days)", "Age 0-12", "Age 12-60", "Age 60+", "Females",
                                     "Baby Food (kg)", "Elder Medicine (units)", "Sanitary Items (units)", "Clothing Supply (sets)"])
    return df

# Generate and save dataset
medicine_clothing_df = generate_medicine_clothing_data()
medicine_clothing_df.to_csv("synthetic_medicine_clothing_data.csv", index=False)

print("Medicine and Clothing dataset generated and saved successfully.")

Medicine and Clothing dataset generated and saved successfully.


# **Model training for predicting medicine and clothing supply**

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

# Load dataset
supply_df = pd.read_csv("synthetic_medicine_clothing_data.csv")

# One-hot encode disaster type
supply_df = pd.get_dummies(supply_df, columns=["Disaster Type"], drop_first=True)

# Define features and target variables
X = supply_df.drop(columns=["Baby Food (kg)", "Elder Medicine (units)", "Sanitary Items (units)", "Clothing Supply (sets)"])
y = supply_df[["Baby Food (kg)", "Elder Medicine (units)", "Sanitary Items (units)", "Clothing Supply (sets)"]]

# Store feature names for later use
feature_names = X.columns

def train_model():
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # Train the model with optimized hyperparameters
    base_model =RandomForestRegressor(n_estimators=750, max_depth=45, min_samples_split=3, min_samples_leaf=1, max_features=0.8, random_state=42)
    model = MultiOutputRegressor(base_model)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R-Squared (R²): {r2:.2f}")

    # Save the trained model and feature names
    with open("supply_model.pkl", "wb") as file:
        pickle.dump((model, feature_names), file)
    print("Model saved successfully.")


# Example usage
train_model()  # Uncomment to train model


Mean Absolute Error (MAE): 172.85
R-Squared (R²): 0.95
Model saved successfully.


# **Model training for predicting food and water**

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

# Load dataset
food_water_df = pd.read_csv("synthetic_food_water_data.csv")

# One-hot encode disaster type
food_water_df = pd.get_dummies(food_water_df, columns=["Disaster Type"], drop_first=True)

# Define features and target variables
X = food_water_df.drop(columns=["Rice Supply (kg)", "Vegetables Supply (kg)", "Dry Food Supply (kg)", "Water Supply (liters)"])
y = food_water_df[["Rice Supply (kg)", "Vegetables Supply (kg)", "Dry Food Supply (kg)", "Water Supply (liters)"]]

# Store feature names for later use
feature_names = X.columns

def train_model():
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # Train the model with optimized hyperparameters
    base_model = RandomForestRegressor(n_estimators=200, max_depth=11, min_samples_split=4, min_samples_leaf=1, max_features='sqrt', random_state=42)
    model = MultiOutputRegressor(base_model)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R-Squared (R²): {r2:.2f}")

    # Save the trained model and feature names
    with open("food_water_model.pkl", "wb") as file:
        pickle.dump((model, feature_names), file)
    print("Model saved successfully.")

# Example usage
train_model()  # Uncomment to train model


Mean Absolute Error (MAE): 59419.63
R-Squared (R²): 0.94
Model saved successfully.


# ***Use this to predict the result***

In [5]:
def predict_supply():
    # Load the trained models
    with open("food_water_model.pkl", "rb") as file:
        food_water_model, food_water_features = pickle.load(file)
    with open("supply_model.pkl", "rb") as file:
        supply_model, supply_features = pickle.load(file)

    # Take user input
    severity = int(input("Enter severity level (1-5): "))
    area_size = float(input("Enter affected area size (sq km): "))
    age_0_12 = int(input("Enter number of affected children (0-12 years): "))
    age_12_60 = int(input("Enter number of affected adults (12-60 years): "))
    age_60_plus = int(input("Enter number of affected elderly (60+ years): "))
    gender_ratio = float(input("Enter gender ratio (male:female) as a number (e.g., 1:1.2 enter 1.2): "))
    duration = int(input("Enter disaster duration (days): "))
    disaster_type = input("Enter disaster type (Flood, Storm, Earthquake, Drought): ")

    # Calculate total population
    population_affected = age_0_12 + age_12_60 + age_60_plus
    females = int((gender_ratio / (1 + gender_ratio)) * population_affected)

    # Prepare input data
    example_input = pd.DataFrame({
        "Severity": [severity],
        "Area Size (sq km)": [area_size],
        "Population Affected": [population_affected],
        "Duration (days)": [duration],
        "Age 0-12": [age_0_12],
        "Age 12-60": [age_12_60],
        "Age 60+": [age_60_plus],
        "Females": [females]
    })

    # Add disaster type encoding
    for dtype in ["Flood", "Storm", "Earthquake", "Drought"]:
        example_input[f"Disaster Type_{dtype}"] = [1 if dtype.lower() == disaster_type.lower() else 0]

    # Ensure all features are present
    food_water_input = example_input.reindex(columns=food_water_features, fill_value=0)
    supply_input = example_input.reindex(columns=supply_features, fill_value=0)

    # Make predictions
    predicted_food_water = food_water_model.predict(food_water_input)[0] * duration
    predicted_supply = supply_model.predict(supply_input)[0] * duration

    print("Predicted Food & Water Supply:")
    print(f"  - Rice Supply: {predicted_food_water[0]:.2f} kg")
    print(f"  - Vegetables Supply: {predicted_food_water[1]:.2f} kg")
    print(f"  - Dry Food Supply: {predicted_food_water[2]:.2f} kg")
    print(f"  - Water Supply: {predicted_food_water[3]:.2f} liters")

    print("\nPredicted Medicine & Clothing Supply:")
    print(f"  - Baby Food: {predicted_supply[0]:.2f} kg")
    print(f"  - Elder Medicine: {predicted_supply[1]:.2f} units")
    print(f"  - Sanitary Items: {predicted_supply[2]:.2f} units")
    print(f"  - Clothing Supply: {predicted_supply[3]:.2f} sets")


predict_supply()  # Uncomment to predict based on user input

Enter severity level (1-5): 4
Enter affected area size (sq km): 1500
Enter number of affected children (0-12 years): 10000
Enter number of affected adults (12-60 years): 80000
Enter number of affected elderly (60+ years): 7000
Enter gender ratio (male:female) as a number (e.g., 1:1.2 enter 1.2): 0.93
Enter disaster duration (days): 10
Enter disaster type (Flood, Storm, Earthquake, Drought): Storm
Predicted Food & Water Supply:
  - Rice Supply: 1605869.87 kg
  - Vegetables Supply: 878973.18 kg
  - Dry Food Supply: 653064.93 kg
  - Water Supply: 15367217.25 liters

Predicted Medicine & Clothing Supply:
  - Baby Food: 50000.00 kg
  - Elder Medicine: 100000.00 units
  - Sanitary Items: 80000.00 units
  - Clothing Supply: 50000.00 sets


# **Model Description**

**Disaster Relief Supply Prediction Model**

### Overview
This model predicts the required quantities of food, water, medicine, and clothing supplies for disaster relief efforts. It utilizes historical data to estimate the supply needs based on disaster type, severity, affected population, and other critical factors.

### Features
- **Multi-Output Regression Model**: Uses RandomForestRegressor wrapped in MultiOutputRegressor to predict multiple supply categories simultaneously.
- **One-Hot Encoding**: Encodes disaster types (Flood, Storm, Earthquake, Drought) for effective model training.
- **Hyperparameter Optimization**: Fine-tuned hyperparameters for improved accuracy.
- **User Input Integration**: Accepts real-time user inputs such as severity level, affected population, gender ratio, and duration to generate supply predictions.
- **Dynamic Population Calculation**: Computes total affected population from different age groups and calculates the number of females based on the gender ratio.
- **Multiplication by Duration**: Ensures the supply predictions are scaled according to the number of days the aid is required.

### Inputs Required
1. **Severity Level (1-5)**: Indicates the intensity of the disaster.
2. **Affected Area Size (sq km)**: Represents the geographical extent of the disaster.
3. **Number of Affected Individuals**: Categorized into:
   - Children (0-12 years)
   - Adults (12-60 years)
   - Elderly (60+ years)
4. **Gender Ratio (Male:Female)**: Converts into an estimated number of affected females.
5. **Disaster Duration (days)**: Multiplies predictions to estimate required supplies over time.
6. **Disaster Type**: Categorical input (Flood, Storm, Earthquake, Drought) that affects supply estimates.

### Predictions
- **Food & Water Supply**:
  - Rice (kg)
  - Vegetables (kg)
  - Dry Food (kg)
  - Water (liters)
- **Medicine & Clothing Supply**:
  - Baby Food (kg)
  - Elder Medicine (units)
  - Sanitary Items (units)
  - Clothing Sets (sets)

### Performance Metrics
- **Food & Water Model**:
  - **Mean Absolute Error (MAE):** 59,419.63
  - **R-Squared (R²):** 0.94
- **Medicine & Clothing Model**:
  - **Mean Absolute Error (MAE):** 172.85
  - **R-Squared (R²):** 0.95

### Model Usage
1. **Training**: The model is trained using a historical dataset containing past disaster relief efforts.
2. **Prediction**: Based on user input, the trained model estimates the required supplies dynamically.

### Conclusion
This model provides a data-driven approach to disaster relief planning, ensuring efficient resource allocation. With its high R² values and optimized parameters, it offers reliable supply predictions for various disaster scenarios.